**Machine learning**

In [1]:
import pandas as pd #librerias necesarias
import numpy as np

In [2]:
df=pd.read_csv(r'C:\Users\Fernando\Desktop\Proyecto 1\Proyecto-Individual-1-ML\data procesada para funciones\data_ml') #imprtamos la data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           45464 non-null  int64  
 1   budget               45376 non-null  float64
 2   Id_Pelicula          45376 non-null  float64
 3   original_language    45365 non-null  object 
 4   overview             44435 non-null  object 
 5   popularity           45376 non-null  float64
 6   release_date         45376 non-null  object 
 7   revenue              45376 non-null  float64
 8   runtime              45130 non-null  float64
 9   status               45296 non-null  object 
 10  tagline              20398 non-null  object 
 11  title                45376 non-null  object 
 12  vote_average         45376 non-null  float64
 13  vote_count           45376 non-null  float64
 14  return               45376 non-null  float64
 15  release_year         45376 non-null 

In [3]:
df=df[['title', 'coleccion', 'overview', 'genero_1', 'genero_2', 'genero_3']] #reducimos el df a las columnas relevantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      45376 non-null  object
 1   coleccion  4488 non-null   object
 2   overview   44435 non-null  object
 3   genero_1   42992 non-null  object
 4   genero_2   28445 non-null  object
 5   genero_3   13972 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [4]:
df=df.fillna('') #llenamos los nulos con '' para que al momento de analizar el lenguaje natural, no se procese nada si hay nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      45464 non-null  object
 1   coleccion  45464 non-null  object
 2   overview   45464 non-null  object
 3   genero_1   45464 non-null  object
 4   genero_2   45464 non-null  object
 5   genero_3   45464 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


Modelo con features **overview, genero 1,2 y 3**

In [5]:
df['combined_features'] = df['overview'] + ' ' + df['genero_1'] + ' ' + df['genero_2'] + ' ' + df['genero_3'] #creamos la columna con lo datos que vamos a usar

In [6]:
df['combined_features'] #vemos los datos

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45459                                       Family Comedy 
45460                               Animation Horror Drama
45461                                        Documentary  
45462                                    Adventure Family 
45463                               Mystery Drama Thriller
Name: combined_features, Length: 45464, dtype: object

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer #impoortamos 

tfidf = TfidfVectorizer(stop_words='english') #instanciamos
matriz_tfidf = tfidf.fit_transform(df['combined_features']) #aplicamos el TF-IDF a la columna de features combinados


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(matriz_tfidf, matriz_tfidf) #calculamos la similitud del coseno entre las 45000 peliculas usando la matriz TF-IDF


In [41]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recomendacion(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]


In [10]:
print(get_recomendacion('Star Wars'))


1152                        The Empire Strikes Back
30400                 The Star Wars Holiday Special
26525                  Star Wars: The Force Awakens
1165                             Return of the Jedi
23787                              Loser Takes All!
36412                                    Hokuspokus
34112    Samson and the Seven Miracles of the World
1265                                   Mad Dog Time
26419                              TekWar: TekLords
22825                                  Scott Joplin
Name: title, dtype: object


Modelo con **overview, genero_1, coleccion**   (Mejor y mas eficiente modelo)

In [11]:
df=pd.read_csv(r'C:\Users\Fernando\Desktop\Proyecto 1\Proyecto-Individual-1-ML\data procesada para funciones\data_ml')


In [30]:
df=df[['Id_Pelicula','title', 'coleccion', 'overview', 'genero_1']]
df=df.fillna('')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id_Pelicula  45464 non-null  object
 1   title        45464 non-null  object
 2   coleccion    45464 non-null  object
 3   overview     45464 non-null  object
 4   genero_1     45464 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [31]:
df['combined_features'] = df['overview'] + ' ' + df['genero_1'] + ' ' + df['coleccion']
df['combined_features']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45459                                              Family 
45460                                           Animation 
45461                                         Documentary 
45462                                           Adventure 
45463                                             Mystery 
Name: combined_features, Length: 45464, dtype: object

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
matriz_tfidf_1 = tfidf.fit_transform(df['combined_features'])

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(matriz_tfidf_1, matriz_tfidf_1)

In [39]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recomendacion(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]


In [43]:
print(get_recomendacion('Star Wars')) #da mejor resultado y es menos intensiva y toma menos tiempo


1152                               The Empire Strikes Back
30400                        The Star Wars Holiday Special
26525                         Star Wars: The Force Awakens
1165                                    Return of the Jedi
15446    Empire of Dreams: The Story of the Star Wars T...
Name: title, dtype: object


**Agregando actores**

In [18]:
df_act=pd.read_csv(r'C:\Users\Fernando\Desktop\Proyecto 1\Proyecto-Individual-1-ML\data procesada para funciones\actores_procesado')
df_act

C:\Users\Fernando\AppData\Local\Temp\ipykernel_18076\704391271.py:1: DtypeWarning: Columns (116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200,202,204,206,208,210,212,214,216,218,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306,308,310,312,314,316,318,320,322,324,326,328,330,332,334,336,338,340,342,344,346,348,350,352,354,356,358,360,362,364,366,368,370,372,374,376,378,380,382,384,386,388,390,392,394,396,398,400,402,404,406,408,410,412,414,416,418,420,422,424,426,428,430,432,434,436,438,440,442,444,446,448,450,452,454,456,458,460,462,464,466,468,470,472,474,476,478,480,482,484,486,488,490,492,494,496,498,500,502,504,506,508,510,512,514,516,518,520,522,524,526,528,530,532,534,536,538,540,542,544,546,548,550,552,554,556,558,560,562,564,566,568,

,Unnamed: 0,Id_Pelicula,Actor 1,Actor 1 Id,Actor 2,Actor 2 Id,Actor 3,Actor 3 Id,Actor 4,Actor 4 Id,...,Actor 309,Actor 309 Id,Actor 310,Actor 310 Id,Actor 311,Actor 311 Id,Actor 312,Actor 312 Id,Actor 313,Actor 313 Id
0,0,862,Tom Hanks,31.0,Tim Allen,12898.0,Don Rickles,7167.0,Jim Varney,12899.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,8844,Robin Williams,2157.0,Jonathan Hyde,8537.0,Kirsten Dunst,205.0,Bradley Pierce,145151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,15602,Walter Matthau,6837.0,Jack Lemmon,3151.0,Ann-Margret,13567.0,Sophia Loren,16757.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,31357,Whitney Houston,8851.0,Angela Bassett,9780.0,Loretta Devine,18284.0,Lela Rochon,51359.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,11862,Steve Martin,67773.0,Diane Keaton,3092.0,Martin Short,519.0,Kimberly Williams-Paisley,70696.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45471,45471,439050,Leila Hatami,240240.0,Kourosh Tahami,1749839.0,Elham Korda,1619957.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45472,45472,111109,Angel Aquino,1043186.0,Perry Dizon,111636.0,Hazel Orencio,1204271.0,Joel Torre,278923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45473,45473,67758,Erika Eleniak,23764.0,Adam Baldwin,2059.0,Julie du Page,46277.0,James Remar,1736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45474,45474,227506,Iwan Mosschuchin,544742.0,Nathalie Lissenko,1090923.0,Pavel Pavlov,1136422.0,Aleksandr Chabrov,1261758.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_act=df_act[['Id_Pelicula','Actor 1','Actor 2','Actor 3','Actor 4','Actor 5']] #agarramos a los 5 primeros ya que son los mas importantes
df_act.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id_Pelicula  45476 non-null  int64 
 1   Actor 1      43058 non-null  object
 2   Actor 2      41726 non-null  object
 3   Actor 3      40816 non-null  object
 4   Actor 4      39683 non-null  object
 5   Actor 5      37207 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.1+ MB


In [20]:
df_act['Id_Pelicula']=df_act['Id_Pelicula'].astype(object) #convertimos a objeto para poder hacer el merge

C:\Users\Fernando\AppData\Local\Temp\ipykernel_18076\1962542561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_act['Id_Pelicula']=df_act['Id_Pelicula'].astype(object) #convertimos a objeto para poder hacer el merge


In [21]:
df_final = pd.merge(df, df_act, on='Id_Pelicula', how='left') #juntamos los dataframes de peliculas con el de actores
df_final.fillna('',inplace=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45540 entries, 0 to 45539
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id_Pelicula        45540 non-null  object
 1   title              45540 non-null  object
 2   coleccion          45540 non-null  object
 3   overview           45540 non-null  object
 4   genero_1           45540 non-null  object
 5   genero_2           45540 non-null  object
 6   genero_3           45540 non-null  object
 7   combined_features  45540 non-null  object
 8   Actor 1            45540 non-null  object
 9   Actor 2            45540 non-null  object
 10  Actor 3            45540 non-null  object
 11  Actor 4            45540 non-null  object
 12  Actor 5            45540 non-null  object
dtypes: object(13)
memory usage: 4.5+ MB


In [22]:
df_final['combined_features']=df_final['title']+''+df_final['coleccion']+''+df_final['genero_1']+''+df_final['Actor 1']+''+df_final['Actor 2']+''+df_final['Actor 3']
df_final['combined_features']

0        Toy StoryToy Story CollectionAnimationTom Hank...
1        JumanjiAdventureRobin WilliamsJonathan HydeKir...
2        Grumpier Old MenGrumpy Old Men CollectionRoman...
3        Waiting to ExhaleComedyWhitney HoustonAngela B...
4        Father of the Bride Part IIFather of the Bride...
                               ...                        
45535                                               Family
45536                                            Animation
45537                                          Documentary
45538                                            Adventure
45539                                              Mystery
Name: combined_features, Length: 45540, dtype: object

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_2 = tfidf.fit_transform(df_final['combined_features'])


In [24]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix_2, tfidf_matrix_2)

In [42]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recomendacion(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]


In [26]:
print(get_recomendacion('Star Wars')) #da peores resultados que solo usando el data mas sencillo de los intentos pasados ya que toma a actores en peliculas poco similares 


1155                        Brazil
1168                    GoodFellas
35856       Vendetta for the Saint
16602                   Torch Song
30466                  Underground
26587           The McKenzie Break
41493                       Fences
2516      The Saragossa Manuscript
12900              American Zombie
15476    The Sorcerer's Apprentice
Name: title, dtype: object


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id_Pelicula        45464 non-null  object
 1   title              45464 non-null  object
 2   coleccion          45464 non-null  object
 3   overview           45464 non-null  object
 4   genero_1           45464 non-null  object
 5   genero_2           45464 non-null  object
 6   genero_3           45464 non-null  object
 7   combined_features  45464 non-null  object
dtypes: object(8)
memory usage: 2.8+ MB
